In [2]:
# Importamos las librerías, si no están instaladas, realizar instalación con pip install en terminal
import cv2
import mediapipe as mp
print("¡Las librerías están correctamente instaladas!")

# Iniciamos MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_dibujo = mp.solutions.drawing_utils

# Capturamos video desde la cámara
captura_video = cv2.VideoCapture(0)

while captura_video.isOpened():
    ret, frame = captura_video.read()
    if not ret:
        break

    # Convertimos la imagen a RGB para MediaPipe
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resultados = pose.process(frame_rgb)

    # Detectamos las poses si se encuentran puntos clave
    if resultados.pose_landmarks:
        mp_dibujo.draw_landmarks(frame, resultados.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        # Obtenemos las coordenadas de los puntos clave
        mu_izq = resultados.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]
        mu_der = resultados.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST]
        cad_izq = resultados.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP]
        cad_der = resultados.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP]

        # Calculamos la altura promedio de las caderas
        avg_alt_cad = (cad_izq.y + cad_der.y) / 2

        # Detectamos si ambas muñecas están por debajo de las caderas (inicio de levantamiento)
        if mu_izq.y > avg_alt_cad and mu_der.y > avg_alt_cad:
            cv2.putText(frame, "Inicio de levantamiento detectado", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Detectamos si ambas muñecas están por encima de las caderas (levantamiento completo)
        elif mu_izq.y < avg_alt_cad and mu_der.y < avg_alt_cad:
            cv2.putText(frame, "Objeto levantado", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        else:
            cv2.putText(frame, "Esperando actividad", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

    # Mostramos el resultado en una ventana
    frame = cv2.resize(frame, (640, 480))
    cv2.imshow("Detectar levantamiento de objeto", frame)

    # Salimos presionando la tecla 'q'
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

captura_video.release()
cv2.destroyAllWindows()



¡Las librerías están correctamente instaladas!
